In [ ]:
import qiskit
qiskit.__version__

> If you have an IBM Quantum Platform, paste in the API token below and run the relevant cells.

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService
service = QiskitRuntimeService(channel = "ibm_quantum", # ibm_cloud
                               token = "...")

In [ ]:
QiskitRuntimeService.save_account(channel = "ibm_quantum", token= "...")

## Hello world example on 2-qubit Bell state

### Generate circuit

In [ ]:
from qiskit import QuantumCircuit
# Generate the |PHI+> Bell state
phi_p = QuantumCircuit(2, 2)  # 2 qubits and 2 bits
phi_p.h(0)
phi_p.cx(0, 1)
phi_p.measure([0, 1], [0, 1])
phi_p.draw(output='mpl')

### Tutorial video method (using estimators with observables)

In [ ]:
from qiskit.quantum_info import Pauli

ZZ = Pauli('ZZ')
ZI = Pauli('ZI')
IZ = Pauli('IZ')
XX = Pauli('XX')
XI = Pauli('XI')
IX = Pauli('IX')

observables = [ZZ, ZI, IZ, XX, XI, IX]

In [ ]:
from qiskit_aer.primitives import Estimator
est = Estimator()
job = est.run([phi_p] * len(observables), observables)
job.result()

In [ ]:
import matplotlib.pyplot as plt

data = ['ZZ', 'ZI', 'IZ', 'XX', 'XI', 'IX']
values = job.result().values

plt.plot(data, values, "-o",)
plt.xlabel("Observables")
plt.ylabel("Expectation values")
plt.show()

### Method using Aer simulator

In [ ]:
from qiskit_aer.primitives import Sampler

sampler = Sampler(backend_options={"seed_simulator": 999})

quasi_dists = sampler.run(phi_p, shots=1000).result().quasi_dists[0].binary_probabilities()
print(quasi_dists)

In [ ]:
from qiskit.visualization import plot_histogram
plot_histogram(quasi_dists)

## Now Extend the Hello world example to $n$-qubit GHZ state

### Making my own version of a $n$-qubit GHZ state

In [ ]:
def get_qc_for_n_qubit_GHZ_state(n):
    qc = QuantumCircuit(n)
    qc.h(0)
    for i in range(1, n):
        qc.cx(0, i)
    return qc

n = 10
qc = get_qc_for_n_qubit_GHZ_state(n)
qc.measure_all()
qc.draw(output='mpl')

#### Measurement, simulate, and read results using a sampler

In [ ]:
from qiskit_aer.primitives import sampler

sampler = Sampler(backend_options={"seed_simulator": 999})

counts = sampler.run(qc, shots=1000).result().quasi_dists[0].binary_probabilities()
print(counts)

### Tutorial video method of generating a GHZ state

In [ ]:
def get_qc_for_n_qubit_GHZ_state2(n):
    qc = QuantumCircuit(n)
    qc.h(0)
    for i in range(n - 1):
        qc.cx(i, i + 1)
    return qc

n = 10
qc = get_qc_for_n_qubit_GHZ_state2(n)
qc.measure_all()
qc.draw(output='mpl')

In [ ]:
from qiskit_aer.primitives import sampler

sampler = Sampler(backend_options={"seed_simulator": 999})

job = sampler.run(qc, shots=1000)
result = job.result()
counts = result.quasi_dists[0].binary_probabilities()
print(counts)

We can see that the two methods of generating the states return the same outcome, so they should be equivalent.

### Tutorial video method

In [ ]:
from qiskit.quantum_info import SparsePauliOp

operator_strings = ['Z' + 'I' * i + 'Z' + 'I' * (n-2-i) for i in range(n-1)]
print(operator_strings)
print(len(operator_strings))

operators = [SparsePauliOp(op_string) for op_string in operator_strings]

Making this $n$-qubit state creation more optimal (Requires IBM remote labs)

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

backend_name = 'ibm_brisbane'
backend = QiskitRuntimeService().get_backend(backend_name)
pass_manager = generate_preset_pass_manager(optimization_level=1, backend=backend)

qc_transpiled = pass_manager.run(qc)
operators_transpiled_list = [op.apply_layout(qc_transpiled.layout) for op in operators]

Execute on backend

In [ ]:
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit_ibm_runtime import EstimatorOptions

options = EstimatorOptions()
options.resilience_level = 1    # Use measurement readout mitigation (e.g. 2 ~ Zero noise extrapolation)
# options.optimization_level = 0  # B/c transpilation already done on local device
options.dynamical_decoupling.enable = True  # Helps disable cross-talk and coherence errors

estimator = Estimator(backend, options=options)

job = estimator.run([(qc_transpiled, operators_transpiled_list)])
job_id = job.job_id()
print(job_id)

Post-process and plot

In [ ]:
service = QiskitRuntimeService()
job = service.job(job_id)

In [ ]:
import matplotlib.pyplot as plt

data = list(range(1, len(operators)+1))
result = job.result()[0]
values = result.data.evs
values = [v / values[0] for v in values]

plt.scatter(data, values, marker="o", label="n-qubit GHZ state")
plt.xlabel("Distance betwwen qubits $i$")
plt.ylabel(r"\langle Z_0 Z_i \rangle / \langle Z_0 Z_1 \rangle$")
plt.legend()
plt.show()